<a href="https://colab.research.google.com/github/IDE21/SDAIA_LLM_Projects/blob/main/Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import time
import csv

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

urls = [
    "https://www.raymond-weil.com/en/collections/watches-en/womens-en/?v=rwrow",
    "https://www.raymond-weil.com/en/collections/watches-en/mens-en/?v=rwrow"
]
product_details = []
driver = webdriver.Chrome()
driver.maximize_window()
wait = WebDriverWait(driver, 10)
count=0
for url in urls:
    try:

        # Open the website
        driver.get(url)
        time.sleep(10) #adjust time as needed
        if count==0:
            webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
            cookie_button = driver.find_element(By.XPATH, '//*[@id="wt-cli-accept-btn"]')
            print(cookie_button.text)
        # cookie_button[0]
            if cookie_button:
                print("cookie_button")
                cookie_button.click()
                time.sleep(2)
    # Scroll to load more content
    # driver.execute_script("window.scrollBy(0, 700);")
        SCROLL_PAUSE_TIME = 5
        last_height = driver.execute_script("return document.body.scrollHeight")
        if count==0:
            for i in range(5):
            # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load more content on the page
                time.sleep(SCROLL_PAUSE_TIME)
                load_more_button = driver.find_element(By.XPATH, '//*[@id="left-area"]/nav/a')
                ActionChains(driver).move_to_element(load_more_button).move_by_offset(0, 150).perform()
                load_more_button.click()
                time.sleep(10)
                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
        # for i in range(5):
        else:
            for i in range(6):
                # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # Wait to load more content on the page
                time.sleep(SCROLL_PAUSE_TIME)
                load_more_button = driver.find_element(By.XPATH, '//*[@id="left-area"]/nav/a')
                ActionChains(driver).move_to_element(load_more_button).move_by_offset(0, 150).perform()
                load_more_button.click()
                time.sleep(10)
                    # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
        getproduct(driver.page_source)
    except Exception as e:
        print(e)

    finally:
        count=count+1
        print("closed")




ACCEPT & CLOSE
cookie_button
closed
closed


In [ ]:
def getproduct(src):


        soup = BeautifulSoup(src, 'html.parser')

        product_items = soup.find_all('li', class_='product')

        for item in product_items:
            product_title = item.find('h2', class_='woocommerce-loop-product__title').text.strip()
            product_link = item.find('a', class_='woocommerce-LoopProduct-link woocommerce-loop-product__link')['href']

            product_details.append({
                'title': product_title,
                'link': product_link
            })



        data=[]
        for product in product_details:
          response1 = requests.get(product['link'], headers=headers)
          if response1.status_code == 200:

            soup1 = BeautifulSoup(response1.text, 'html.parser')
          else:
            print(f"Failed to retrieve product details for {product['title']}. Status Code: {response1.status_code}")

          reference_number_element = soup1.find(class_='rwproduct__main__infos__sku')
          reference_number = reference_number_element.text.strip() if reference_number_element else ""
          watch_URL =product['link']


          type_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Size') + td")
          type_value = type_element.get_text(strip=True) if type_element else ""
          brand = "Raymond Weil"
          year_introduced_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Introduced') + td")
          year_introduced = year_introduced_element.get_text(strip=True) if year_introduced_element else ""

          parent_model_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Collection') + td")
          parent_model = parent_model_element.get_text(strip=True) if parent_model_element else ""

          specific_model_element = soup1.find(class_='rwproduct__main__infos__name')
          specific_model = specific_model_element.find("div").text.strip() if specific_model_element else ""

          nickname_element = soup1.find(class_='rwproduct__main__infos__name')
          nickname = nickname_element.find("div").text.strip() if nickname_element else ""

          marketing_name_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Model') + td")
          marketing_name = marketing_name_element.get_text(strip=True) if marketing_name_element else ""

          style_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Style') + td")
          style = style_element.get_text(strip=True) if style_element else ""

          currency = 'USD'

          price_element = soup1.find(class_='woocommerce-Price-amount amount')
          price = price_element.text.strip() if price_element else ""

          image_elements = soup1.find_all("div", {"class": "woocommerce-product-gallery__wrapper"})
          image_url = image_elements[0].find("img").attrs["data-src"] if image_elements else ""

          made_in = "Switzerland"

          case_shape_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Shape') + td")
          case_shape = case_shape_element.get_text(strip=True) if case_shape_element else ""

          case_material_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('material') + td")
          case_material = case_material_element.get_text(strip=True) if case_material_element else ""

          case_finish_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Finish') + td")
          case_finish = case_finish_element.get_text(strip=True) if case_finish_element else ""

          caseback_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Case back') + td")
          caseback = caseback_element.get_text(strip=True) if caseback_element else ""

          diameter_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Case size') + td")
          diameter = diameter_element.get_text(strip=True) if diameter_element else ""

          between_lugs=" "
          lug_to_lug=" "

          case_thickness_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('thickness') + td")
          case_thickness = case_thickness_element.get_text(strip=True) if case_thickness_element else ""

          bezel_material_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Bezel') + td")
          bezel_material = bezel_material_element.get_text(strip=True) if bezel_material_element else ""

          bezel_color_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Bezel') + td")
          bezel_color = bezel_color_element.get_text(strip=True) if bezel_color_element else ""

          crystal_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Crystal') + td")
          crystal = crystal_element.get_text(strip=True) if crystal_element else ""

          water_resistance_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Water') + td")
          water_resistance = water_resistance_element.get_text(strip=True) if water_resistance_element else ""

          weight_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Weight') + td")
          weight = weight_element.get_text(strip=True) if weight_element else ""

          dial_color_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Dial') + td")
          dial_color = dial_color_element.get_text(strip=True) if dial_color_element else ""

          numerals_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Dial') + td")
          numerals = numerals_element.get_text(strip=True) if numerals_element else ""

          bracelet_material_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Bracelet/Strap') + td")
          bracelet_material = bracelet_material_element.get_text(strip=True) if bracelet_material_element else ""

          bracelet_color_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Bracelet/Strap') + td")
          bracelet_color = bracelet_color_element.get_text(strip=True) if bracelet_color_element else ""

          clasp_type_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Clasp') + td")
          clasp_type = clasp_type_element.get_text(strip=True) if clasp_type_element else ""

          movement_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Movement') + td")
          movement = movement_element.get_text(strip=True) if movement_element else ""

          caliber_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Movement') + td")
          caliber = caliber_element.get_text(strip=True) if caliber_element else ""

          power_reserve_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Power reserve') + td")
          power_reserve = power_reserve_element.get_text(strip=True) if power_reserve_element else ""

          frequency_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Frequency') + td")
          frequency = frequency_element.get_text(strip=True) if frequency_element else ""

          jewels_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Jewels') + td")
          jewels = jewels_element.get_text(strip=True) if jewels_element else ""

          features_element = soup1.select_one("table.rwproduct__datas__tech__table tr th:contains('Features') + td")
          features = features_element.get_text(strip=True) if features_element else ""

          description_element = soup1.select_one("div.rwproduct__datas__description__inner")
          description_text = description_element.text.strip() if description_element else ""

          start_index = description_text.index("Description") + len("Description")
          description = description_text[start_index:].strip()

          short_description_element = soup1.find('span', {'data-sheets-value': True})
          short_description = short_description_element.text.strip() if short_description_element else ""


          data.append([reference_number,watch_URL,type_value, brand, year_introduced, parent_model, specific_model, nickname,
                     marketing_name, style, currency, price, image_url, made_in, case_shape, case_material,
                     case_finish, caseback, diameter, between_lugs, lug_to_lug, case_thickness, bezel_material,
                     bezel_color, crystal, water_resistance, weight, dial_color, numerals, bracelet_material,
                     bracelet_color, clasp_type, movement, caliber, power_reserve, frequency, jewels, features,
                     description, short_description])



        with open('raymond_weil_project.csv', mode='w', newline='',encoding='utf-8') as file:
          writer = csv.writer(file)
          writer.writerow(['Reference_Number','watch_URL','type', 'brand', 'year_introduced', 'parent_model', 'specific_model',
                     'nickname', 'marketing_name', 'style', 'currency', 'price', 'image_url', 'made_in',
                     'case_shape', 'Case_material', 'case_finish', 'caseback', 'diameter', 'between_lugs',
                     'lug_to_lug', 'Case_thickness', 'bezel_material', 'bezel_color', 'crystal', 'water_resistance',
                     'weight', 'dial_color', 'numerals', 'bracelet_material', 'bracelet_color', 'clasp_type',
                     'movement', 'caliber', 'power_reserve', 'frequency', 'jewels', 'features', 'description',
                     'short_description'])
          writer.writerows(data)


